# Rat tumors - hierarchical modeling

In [1]:
import pymc as pm

In [2]:
samples = [
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    19,
    19,
    19,
    19,
    18,
    18,
    17,
    20,
    20,
    20,
    20,
    19,
    19,
    18,
    18,
    25,
    24,
    23,
    20,
    20,
    20,
    20,
    20,
    20,
    10,
    49,
    19,
    46,
    27,
    17,
    49,
    47,
    20,
    20,
    13,
    48,
    50,
    20,
    20,
    20,
    20,
    20,
    20,
    20,
    48,
    19,
    19,
    19,
    22,
    46,
    49,
    20,
    20,
    23,
    19,
    22,
    20,
    20,
    20,
    52,
    47,
    46,
    24,
]

counts = [
    *[0 for _ in range(10)],
    *[0 for _ in range(4)],
    *[1 for _ in range(6)],
    1,
    1,
    *[2 for _ in range(8)],
    *[2, 1, 5, 2, 5, 3, 2, 7, 7, 3],
    *[3, 2, 9, 10],
    *[4 for _ in range(6)],
    *[4, 10, 4, 4, 4, 5, 11, 12, 5, 5],
    *[6, 5, 6, 6, 6, 6, 16, 15, 15, 9],
]

Model:
$$
\begin{align}
p(\alpha, \beta) & \propto (\alpha + \beta)^{-5/2}\\
\theta_j & \sim \text{Beta}(\alpha, \beta)\\
y_j & \sim \text{Binomial}(n_j, \theta_j) 
\end{align}
$$
- the hyperprior distribution comes from the fact that an uniform prior on:
$$
\left( \frac{\alpha}{\alpha 
+ \beta}, (\alpha + \beta)^{-1/2} \right)
$$
gives the above prior distribution when multiplied by the Jacobian

In [3]:
coords = {"experiment": [f"Experiment {i + 1}" for i in range(len(samples))]}

with pm.Model(coords=coords) as m:
    u = pm.Uniform("u", lower=0, upper=1)
    v = pm.Uniform("v", lower=0, upper=1)
    # solved u = a / (a + b) and v = (a + b)^(-1/2) for a and b
    alpha = pm.Deterministic("alpha", u / (v**2))
    beta = pm.Deterministic("beta", (1 - u) / (v**2))
    pm.Deterministic("log_ratio", pm.math.log(alpha / beta))
    pm.Deterministic("log_sum", pm.math.log(alpha + beta))
    thetas = pm.Beta("theta", alpha=alpha, beta=beta, dims="experiment")
    pm.Binomial("tumors", n=samples, p=thetas, observed=counts, dims="experiment")

    trace = pm.sample(2000, tune=1000)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [u, v, theta]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 4 seconds.
